In [5]:
# Question: Data Quality Automation Tools - Introduction to Great Expectations
# Description: Set up a simple Great Expectations check for missing values in a numeric column.



In [6]:
import great_expectations as ge
import pandas as pd

# Sample dataframe
data = {
    "age": [25, 30, None, 22, 40],
    "salary": [50000, 60000, 55000, None, 70000]
}
df = pd.DataFrame(data)

# Convert to Great Expectations dataframe
ge_df = ge.from_pandas(df)

# Check for missing values in the 'age' column
result = ge_df.expect_column_values_to_not_be_null("age")

# Print the result
print(result)


AttributeError: module 'great_expectations' has no attribute 'from_pandas'

In [ ]:
import great_expectations as ge
import pandas as pd

# Sample dataframe
data = {
    "age": [25, 30, None, 22, 40],
    "salary": [50000, 60000, 55000, None, 70000]
}
df = pd.DataFrame(data)

# Convert to Great Expectations dataframe
ge_df = ge.from_pandas(df)

# Check for missing values in the 'age' column
result = ge_df.expect_column_values_to_not_be_null("age")

# Print the result
print(result)
